In [1]:
import requests
import pandas as pd
import random
import time

CVR_VIRK_URL = 'http://distribution.virk.dk/cvr-permanent/virksomhed/_search'
CVR_PROD_URL = 'http://distribution.virk.dk/cvr-permanent/produktionsenhed/_search'
CVR_PROD_SCROLL = 'http://distribution.virk.dk/cvr-permanent/produktionsenhed/_search?scroll=1m'
CVR_SCROLL = 'http://distribution.virk.dk/_search/scroll'

In [7]:
auth_bruger = ''
auth_pass = ''
auth = (auth_bruger, auth_pass)

In [8]:
HEADERS = {
    'Content-Type': 'application/json',
}

FIELDS = ["Vrvirksomhed.cvrNummer",
          "Vrvirksomhed.deltagerRelation"]

In [9]:
def get_orgs(cvr, auth, fields = FIELDS, api_endpoint = CVR_VIRK_URL, headers = HEADERS): # Function for getting company info
    query_body = {"_source": fields,
                  "query": {
                      "bool": {
                          "must": [
                              {"match": {"Vrvirksomhed.cvrNummer": cvr}}
                          ]
                      }
                  }
                 }
    
    
    query = str(query_body).replace('\'', '\"')
    
    response = requests.post(api_endpoint, headers = headers, auth = auth, data = query)
    
    data_response = dict(response.json())

    #data = pd.json_normalize(data_response['hits']['hits'])
    
    return(data_response)

In [10]:
test_cvr = "10010861" # cvr for testing (has to be string because that makes total sense for a CVR-number)

In [11]:
company = get_orgs(test_cvr, auth = auth) # getting company data

In [12]:
orgs = company.get('hits').get('hits')[0].get('_source').get('Vrvirksomhed').get('deltagerRelation') # getting data from field "deltagerRelation" - data regarding associated companies

In [13]:
revifirms = [] # Extracting accounting firms ('hovedtype': 'REVISION')

for org in orgs:
    if 'REVISION' in [orgnavn.get('hovedtype') for orgnavn in org.get('organisationer')]:
        revifirms.append(org)

In [16]:
revifirms[0].get('deltager').get('forretningsnoegle') # First entry seems to be current company (can be checked further with date info)

10092159